In [ ]:
import duckdb
db = duckdb.connect()
db.install_extension("h3", repository = "community")
db.close()

In [ ]:
import ibis
from ibis import _
con = ibis.duckdb.connect(extensions=['httpfs', 'spatial', 'h3'])
gbif = con.read_parquet("../gbif_hex/**")


In [ ]:
# enable ibis to use built-in function from the h3 extension
@ibis.udf.scalar.builtin
def h3_cell_to_boundary_wkt	(array) -> str:
    ...

In [ ]:
# Configure write-access to source.coop
import streamlit as st
source_key = st.secrets["SOURCE_KEY"]
source_secret = st.secrets["SOURCE_SECRET"]

query=   f'''
CREATE OR REPLACE SECRET secret1 (
    TYPE S3,
    KEY_ID '{source_key}',
    SECRET '{source_secret}',
    ENDPOINT 'data.source.coop',
    URL_STYLE 'path'

);
'''
con.raw_sql(query)

In [ ]:
(gbif
 # .filter(_["class"] == "Mammalia")
  .filter(_.species == "Megaptera novaeangliae")
  .rename(hex = "h7")
  .group_by(_.hex)
  .agg(n = _.count())
#  .mutate(wkt =  h3_cell_to_boundary_wkt(_.hex))
  .mutate(logn = _.n.log())
  .mutate(value = (255 * _.logn / _.logn.max()).cast("int"))
  .to_csv("s3://cboettig/gbif/csv/gbif_hex_humpback_whale.csv")
)


In [ ]:
import pydeck as pdk
import pandas as pd

COUNTRIES = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_scale_rank.geojson"
GBIF = "https://data.source.coop/cboettig/gbif/csv/gbif_hex_humpback_whale.csv"

view_state = pdk.ViewState(latitude=51.47, longitude=0.45, zoom=0)
view = pdk.View(type="_GlobeView", controller=True, width=1000, height=600)


layers = [
    pdk.Layer(
        "GeoJsonLayer",
        id="base-map",
        data=COUNTRIES,
        stroked=False,
        filled=True,
        get_fill_color=[200, 200, 200],
    ),
    pdk.Layer(
        "H3HexagonLayer",
        id="gbif",
        data=GBIF,
        extruded=True,
        get_elevation="value",
        get_hexagon="hex",
        elevation_scale=500,
        elevation_range = [0,1],
        pickable=True,
        auto_highlight=True,
        get_fill_color="[255 - value, 255, value]",
    ),
]

deck = pdk.Deck(
    views=[view],
    initial_view_state=view_state,
    layers=layers,
    map_provider=None,
    # Note that this must be set for the globe to be opaque
    parameters={"cull": True},
)
deck.to_html("gbif_3d_humpback_whale.html")